In [ ]:
! pip install gradio_client
! pip install groq
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
import itertools
import re
import numpy as np
import pandas as pd
# import openai
import pandas as pd
import itertools
# import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client
from transformers import pipeline
from groq import Groq


In [ ]:
df = pd.read_parquet('Halved-DF.parquet.gzip')

In [ ]:
def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    '''

    ingredients_combinations = []
    powerpowerset = []
    ingredients = re.split(r',', ingredients_input.strip())
    ingredients_list = list(set(ingredient.strip() for ingredient in ingredients))
    for r in range(len(ingredients_list)+1):
        combinations = itertools.combinations(ingredients_list, r)
        #powerset.extend(subset for subset in combinations if len(subset) > 1)
        for comb in combinations:
            if len(comb) > 1:
                if len(comb) < 3:
                    ingredients_combinations.append(comb)
                else:
                    powerpowerset.append(comb)
                    for power in powerpowerset:
                        lowerset = []
                        combins = itertools.combinations(power, 2)
                        for arrange in combins:
                            lowerset.append(arrange)
                    ingredients_combinations.append(lowerset)
    return ingredients_combinations

In [ ]:
def data_query(df, ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})

    df_comb = pd.DataFrame(data)
    return df_comb

In [ ]:
def muse_comb(data_query_df): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     the function calculates the sum of the "Score" values and returns the three combinations with the largest sums
     OUTPUT: [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(data_query_df)):
        data_query_df["Sum"] = data_query_df["Score"].apply(calculate_sum)

    max_values = data_query_df.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists


In [ ]:
def recipe_generator(lists):
    api_key = "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"
    client = Groq(
    api_key=api_key
    )
    recipe_list = []

    if len(lists) == 1:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {lists[0]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    else:
      for i in range(len(lists)):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {lists[0]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    return recipe_list


In [ ]:
def final_recipes(recipe_dict, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    INPUT: Output of the recipe generator function
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
    """
    final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
    threshold = 0.5

    for i in range(len(recipe_dict)):
        if scores[i] >= threshold:
            final_recipes["Title"].append(recipe_dict[i]['title'])
            final_recipes["Ingredients"].append(recipe_dict[i]['ingredients'])
            final_recipes["Directions"].append(recipe_dict[i]['directions'])
        else:
            n = 0
            tmp_recipe = {
                "Title":recipe_dict[i]['title'],
                "Ingredients":recipe_dict[i]['ingredients'],
                "Directions":recipe_dict[i]['directions']
                         }
            last_recipe = {"title":[],
                           "ingredients":[],
                           "directions":[]
                         }
            while n < 3:
                new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
                new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
                if new_score[0][1] >= threshold:
                    final_recipes["Title"].append(new_recipe[0]["title"])
                    final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
                    final_recipes["Directions"].append(new_recipe[0]["directions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe["title"][0])
                final_recipes["Ingredients"].append(last_recipe["ingredients"][0])
                final_recipes["Directions"].append(last_recipe["directions"][0])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

In [ ]:
def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model

model = get_model()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For 

In [ ]:
combinations = combinations_of_two('chicken, mustard, coconut, orange, gravy')

In [ ]:
df_comb = data_query(df, combinations)
df_comb

,Combination,Score
0,"(chicken, mustard)",[1.0066298927435484]
1,"(chicken, orange)",[1.0035599592711963]
2,"(chicken, coconut)",[1.0035599592711963]
3,"(chicken, gravy)",[1.0004900257988445]
4,"(mustard, orange)",[1.0036608667953153]
5,"(mustard, coconut)",[1.001709310046983]
6,"(mustard, gravy)",[1.0192917213569856]
7,"(orange, coconut)",[1.0069021295275749]
8,"[(chicken, mustard), (chicken, orange), (musta...","[1.0066298927435484, 1.0035599592711963, 1.003..."
9,"[(chicken, mustard), (chicken, coconut), (must...","[1.0066298927435484, 1.0035599592711963, 1.001..."


In [ ]:
lists = muse_comb(df_comb)
lists

[['orange', 'mustard', 'coconut', 'chicken'],
 ['mustard', 'gravy', 'chicken'],
 ['orange', 'coconut', 'chicken']]

In [ ]:
recipe_dicts = recipe_generator(lists)
recipe_dicts

[{'title': 'Orange Glazed Chicken with Mustard and Coconut Cream',
  'ingredients': '* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 1/4 cup coconut cream\n* 2 tbsp Dijon mustard\n* 2 tbsp freshly squeezed orange juice\n* 1 tsp honey\n* Salt and pepper to taste\n* Fresh orange slices and chopped cilantro for garnish (optional)',
  'directions': '1. Preheat oven to 400°F (200°C).\n2. In a small bowl, whisk together coconut cream, mustard, orange juice, and honey until smooth.\n3. In a separate bowl, season chicken with salt and pepper.\n4. In a large skillet, heat the coconut cream mixture over medium heat. Add chicken and cook until browned on both sides, about 5-6 minutes.\n5. Transfer the skillet to the preheated oven and bake for 15-20 minutes or until chicken is cooked through.\n6. Serve hot, garnished with fresh orange slices and chopped cilantro if desired.'},
 {'title': 'Sunset Chicken with Orange-Mustard Glaze and Toasted Coconut Flakes',
  'ingredients': '* 4 bonele

In [ ]:
scores = []
recipe_direction = []

for recipe in recipe_dicts:
    if 'directions' in recipe:
        recipe_direction.append(recipe['directions'])
    else:
        recipe_direction.append("")

for direction in recipe_direction:
    scores.append(model.predict_proba([direction])[0][1])

print(scores)

[0.5190377152367373, 0.49590262229931037, 0.5044214773198488]


In [ ]:
main_dict = final_recipes(recipe_dicts, scores, model)
print(main_dict)

{'Title': ['Orange Glazed Chicken with Mustard and Coconut Cream', 'Grilled Orange Chicken with Coconut Crust', 'Orange Miso Chicken with Coconut Glaze'], 'Ingredients': ['* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 1/4 cup coconut cream\n* 2 tbsp Dijon mustard\n* 2 tbsp freshly squeezed orange juice\n* 1 tsp honey\n* Salt and pepper to taste\n* Fresh orange slices and chopped cilantro for garnish (optional)', '* 4 boneless, skinless chicken breasts\n* 1/2 cup freshly squeezed orange juice\n* 2 tablespoons Dijon mustard\n* 1/4 cup shredded toasted coconut flakes\n* Salt and pepper, to taste', '* 1 lb chicken breast or thighs\n* 2 tbsp orange juice\n* 1 tsp mustard\n* 1/4 cup coconut oil\n* 2 tbsp shredded coconut\n* Salt and pepper to taste'], 'Directions': ['1. Preheat oven to 400°F (200°C).\n2. In a small bowl, whisk together coconut cream, mustard, orange juice, and honey until smooth.\n3. In a separate bowl, season chicken with salt and pepper.\n4. In a large skillet